# Item to Item

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.cluster as clu
from sklearn import neighbors as nei
from sklearn import metrics as met

In [ ]:
df = pd.read_csv("train.csv")
df.head()

,userID,movieID,rating
0,4490,2109,4
1,5839,3471,4
2,5382,150,3
3,1262,1237,5
4,6005,2273,4


In [ ]:
train_size = round(df.shape[0]*0.999)
test_size = df.shape[0] - round(df.shape[0]*0.999) 
print("train size", train_size)
print("test size", test_size)

train_df = df[:train_size]
print(train_df.shape)
test = df[train_size:]
print(test.shape)

train size 989219
test size 990
(989219, 3)
(990, 3)


In [ ]:
test = df[train_size:]
test.head()

,userID,movieID,rating
989219,5847,344,4
989220,5626,3527,3
989221,2798,1544,3
989222,4969,1617,5
989223,4672,3175,5


In [ ]:
U = train_df.pivot("movieID","userID","rating")
U.head()

userID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6014,6015,6016,6017,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
movieID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,3.0,NaN,NaN,NaN,NaN,4.0,5.0,NaN,5.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,NaN,4.0,NaN,3.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
U.shape

(3705, 6040)

In [ ]:
train = U.copy()

In [ ]:
myPreds = []
realLabels = []
t = 0
for i in test.index:
    k = 50
    movieId = test.movieID[i] 
    userId  = test.userID[i]
    filtered_train = train.dropna(subset=[userId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)
    #print(filtered_train.head())
    print(movieId,userId)

    if k >= len(filtered_train):
        k = len(filtered_train)

    nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
    nn.fit(filtered_train)
    
    test_row = U.loc[movieId:movieId, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]

    movieIds = filtered_train.iloc[indices, :].index
    pred = train.loc[movieIds,userId].values.mean()
    if pred > 5:
        pred = 5
    if pred < 1:
        pred = 1  
        
    myPreds.append(pred)
    real_label = test.rating[i]
    print(pred,real_label)
    realLabels.append(real_label)


344 5847
4.02 4
3527 5626
3.62 3
1544 2798
3.64 3
1617 4969
4.14 5
3175 4672
3.76 5
1962 4147
4.46 4
1212 3534
4.68 4
3083 965
3.42 3
421 3107
3.28 3
2572 103
3.5 2
2337 5202
3.98 4
653 41
2.98 3
296 2522
3.38 2
20 3184
2.92 3
50 2529
4.04 4
3462 2495
3.72 4
2411 3665
2.7 2
2581 3361
3.02 2
919 5278
3.739130434782609 4
2795 5252
3.54 5
2253 4987
3.74 4
3698 2641
3.82 5
2460 5492
3.18 1
1722 708
3.78 4
1527 1906
4.28 5
2443 4144
3.42 2
2058 4152
3.24 2
2483 821
3.04 2
1370 276
3.1 3
539 654
3.04 2
3794 3797
3.62 2
3683 1826
4.08 3
1625 2986
3.74 4
3703 2685
4.42 4
1405 1309
3.38 3
1573 5222
3.22 3
1042 3500
3.5 3
1965 4287
2.9285714285714284 5
1304 1927
3.54 5
707 2233
2.96 2
2002 4009
3.88 5
3173 3920
3.58 5
2432 4121
3.3 2
3033 2861
2.44 3
3114 2540
4.18 5
260 5979
3.36 4
350 5020
4.7 3
1080 566
4.64 5
372 5599
3.52 3
2160 4334
4.42 5
2427 3770
4.36 3
1387 2676
4.42 5
587 5959
4.26 5
1278 428
3.74 4
1291 690
3.88 4
3111 821
3.52 3
923 456
4.36 4
2890 5039
4.32 4
3623 4948
3.48 1
587 5

In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.9548209690482192

In [ ]:
test = pd.read_csv("test_without_labels.csv",sep="_",names=["userID","movieID"])[1:]
train = U.copy()
test.head(5)


,userID,movieID
1,629,2683.0
2,601,904.0
3,5673,3717.0
4,5616,1721.0
5,1099,3697.0


In [ ]:
myPreds = []
realLabels = []
movieIds = []
userIds = []
user_and_movieID = []
t = 0
for i in test.index:
    k = 50
    movieId = int(test.movieID[i] )
    userId  = int(test.userID[i] )
    user_and_movieID.append(str(userId) + "_" + str(movieId))

    filtered_train = train.dropna(subset=[userId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)
    if k >= len(filtered_train):
        k = len(filtered_train)
        
    nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
    nn.fit(filtered_train)

    test_row = U.loc[movieId:movieId, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]

    movieIds = filtered_train.iloc[indices, :].index
    pred = train.loc[movieIds,userId].values.mean()

    if pred > 5:
        pred == 5
    if pred < 1:
        pred = 1  
    myPreds.append(pred)

    t += 1
    if t % 500 == 0:
        print(t)
        

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [ ]:
pd.DataFrame( { "IDs": user_and_movieID,
                "rating" : myPreds}).to_csv("submission3.csv",index=False) # RMSE =  0.91553

# Item to Item with weighted averages

In [ ]:
train_size = round(df.shape[0]*0.999)
test_size = df.shape[0] - round(df.shape[0]*0.999) 
print("train size", train_size)
print("test size", test_size)

train_df = df[:train_size]
print(train_df.shape)
test = df[train_size:]
print(test.shape)

train size 989219
test size 990
(989219, 3)
(990, 3)


In [ ]:
myPreds = []
realLabels = []
t = 0
for i in test.index:
    k = 50
    movieId = test.movieID[i] 
    userId  = test.userID[i]
    filtered_train = train.dropna(subset=[userId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)
    #print(filtered_train.head())
    print(movieId,userId)

    if k >= len(filtered_train):
        k = len(filtered_train)

    nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
    nn.fit(filtered_train)
    
    test_row = U.loc[movieId:movieId, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]
    similarities = nn.kneighbors(test_row, return_distance=True)[1]

    movieIds = filtered_train.iloc[indices, :].index
    pred = (np.array(train.loc[movieIds,userId].values) * np.array(similarities)).sum() / (np.array(similarities)).sum()
    if pred > 5:
        pred = 5
    if pred < 1:
        pred = 1  
        
    myPreds.append(pred)
    real_label = test.rating[i]
    print(pred,real_label)
    realLabels.append(real_label)


344 5847
4.0849571317225255 4
3527 5626
3.6217691443249316 3
1544 2798
3.599155227032735 3
1617 4969
3.942041908158716 5
3175 4672
3.775923333692258 5
1962 4147
4.415040561280421 4
1212 3534
4.561253405994551 4
3083 965
3.351907255048616 3
421 3107
3.2938589346799154 3
2572 103
3.425896762904637 2
2337 5202
3.907817269952028 4
653 41
2.9330607476635513 3
296 2522
3.374535177748029 2
20 3184
2.8429874957756 3
50 2529
3.947845804988662 4
3462 2495
3.7435214767483136 4
2411 3665
2.93892168437623 2
2581 3361
3.078350883455292 2
919 5278
3.3873517786561265 4
2795 5252
3.475982532751092 5
2253 4987
3.8063961485557085 4
3698 2641
3.927854347456185 5
2460 5492
3.015827164951556 1
1722 708
3.794151347581328 4
1527 1906
4.346308724832215 5
2443 4144
3.339272503082614 2
2058 4152
3.2713994565217392 2
2483 821
3.010852250099562 2
1370 276
2.982178577923259 3
539 654
3.0524961399897066 2
3794 3797
3.40536013400335 2
3683 1826
4.103232817169247 3
1625 2986
3.7157573998839233 4
3703 2685
4.3001686340

In [ ]:
met.mean_squared_error(realLabels, myPreds, squared=False)

0.9540952316835422

In [ ]:
test = pd.read_csv("test_without_labels.csv",sep="_",names=["userID","movieID"])[1:]
train = U.copy()
test.head(5)


,userID,movieID
1,629,2683.0
2,601,904.0
3,5673,3717.0
4,5616,1721.0
5,1099,3697.0


In [ ]:
myPreds = []
realLabels = []
movieIds = []
userIds = []
user_and_movieID = []
t = 0
for i in test.index:
    k = 50
    movieId = int(test.movieID[i] )
    userId  = int(test.userID[i] )
    user_and_movieID.append(str(userId) + "_" + str(movieId))

    filtered_train = train.dropna(subset=[userId])
    filtered_train = filtered_train - filtered_train.mean(axis=1).values.reshape(-1,1)
    filtered_train.fillna(0, inplace=True)
    if k >= len(filtered_train):
        k = len(filtered_train)
        
    nn = nei.NearestNeighbors(n_neighbors=k,metric="correlation")
    nn.fit(filtered_train)

    test_row = U.loc[movieId:movieId, :]
    test_row = test_row - test_row.mean(axis=1).values.reshape(-1,1)
    test_row.fillna(0, inplace=True)

    indices = nn.kneighbors(test_row, return_distance=False)[0]
    similarities = nn.kneighbors(test_row, return_distance=True)[1]

    movieIds = filtered_train.iloc[indices, :].index
    pred = (np.array(train.loc[movieIds,userId].values) * np.array(similarities)).sum() / (np.array(similarities)).sum()

    if pred > 5:
        pred == 5
    if pred < 1:
        pred = 1  
    myPreds.append(pred)

    t += 1
    if t % 500 == 0:
        print(t)
        

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500


In [ ]:
pd.DataFrame( { "IDs": user_and_movieID,
                "rating" : myPreds}).to_csv("submission4.csv",index=False) # RMSE =  0.92276 didnot improve